# 3 word2vec

## 3.4 CBOWモデルの実装

CBOWの実装を行う。
実装するニューラルネットワークは図3-19．
SimpleCBOWという名前で実装する（次章では、これを改良したCBOWクラスを実装）


In [ ]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul,SoftmaxWithLoss

In [ ]:
class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):  # hidden_sizeは中間層のニューロン数
        V, H = vocab_size, hidden_size

        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # レイヤの生成
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # すべての重みと勾配をリストにまとめる
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in

    def forward(self, contexts, target):
        # contextsは3次元のNumpy配列で図3-18でいうと形状(6,2,7)の3次元配列
        # 0番目の次元：要素数はミニバッチの数
        # 1番目の次元：要素数はコンテキストのウィンドウサイズ分（両側あるのでwindow_sizeの2倍）
        # 2番目の次元：one-hot化されたベクトル
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss

    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None

注：同じ重み（W_in）を２つのMatMulレイヤーで共有していることに注意。
params配列には同じ重みが複数存在し、AdamやMomentumなどのオプティマイザの処理が本来の挙動と異なってしまう。
Trainerクラスの内部ではパラメータの重複を取り除く（common/trainer.pyのremove_duplicate参照）。

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')  # 親ディレクトリのファイルをインポートするための設定
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import preprocess, create_contexts_target, convert_one_hot


window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)

In [ ]:
trainer.plot()

In [ ]:
word_vecs = model.word_vecs
for word_id, word in id_to_word.items():
    print(word, word_vecs[word_id])

## 3.5 word2vecに関する補足

### 3.5.1 CBOWモデルと確率

P(A):Aという事象が起きる確率
P(A,B):AとBが同時に起こる確率
P(A|B):事後確率（Bという情報が与えられたときにAが起こる確率）

w_1,w_2,...,w_Tという単語の列で表されるコーパスを考える。
そして図3-22のようにt番目の単語に対して、ウィンドウサイズが1のコンテキストを考える。

コンテキストとしてw_{t-1}とw_{t+1}が与えられたときに、
ターゲットがw_tとなる確率を数式で表すと

P(w_t|w_{t-1},w_{t+1})

CBOWモデルの損失関数は、この確率に対してlogを取りマイナスを付けたものになる
（交差エントロピーを最小化すること、one-hot化されていることを考える）。

L = -log P(w_t|w_{t-1},w_{t+1})

これは負の対数尤度と呼ばれる。
これは１つのサンプルデータに関する損失関数であり、コーパス全体に拡張すると、損失関数は次のように書ける。

L = -1/T \sum_{t=1}^T log P(w_t|w_{t-1},w_{t+1})

CBOWモデルの学習で行うことは、上の損失関数をできる限り小さくすることである。

### 3.5.2 skip-gramモデル

word2vecでは2つのモデルが提案されている。
ひとつはCBOWモデル、もうひとつがskip-gram。

skip-gramはCBOWで扱うコンテキストとターゲットを逆転させたモデル（図3-23参照）。
skip-gramモデルのネットワーク構成は図3-24のようになる。

確率の表記を使うと、skip-gramは

P(w_{t-1}, w_{t+1}|w_t)

をモデル化する。
skip-gramモデルではコンテキストの単語の間に関連性がないこと（条件付き独立）を仮定し、次のように分解する。

P(w_{t-1}, w_{t+1}|w_t) = P(w_{t-1}|w_t) P(w_{t+1}|w_t)

### 3.5.3 カウントベース v.s. 推論ベース

語彙に新しい単語を追加するとき、単語の分散表現を修正するとき
 - カウントベース：ゼロから計算しなおし
 - 推論ベース（word2vec）：これまでの学習結果を初期値として再学習可能

得られる単語の分散表現の性質
 - カウントベース：単語の類似性
 - 推論ベース：単語の類似性に加えて、複雑な単語間パターン（king - man + woman = queenのような類推問題を解ける）。
 
精度については優劣がつけられない。

推論ベースの手法とカウントベースの手法には関連性があることが分かっている。
具体的にはskip-gramと（次章で扱う）Negative Samplingを利用したモデルは、
コーパス全体の共起行列に対して特殊な行列分解をしているのと同じであることが示されている。
つまり、２つの世界はある条件において「つながっている」。

さらにword2vec以降、推論ベースとカウントベースの手法を融合させたようなGloVeという手法も提案されている。
その手法のアイデアは、コーパス全体の統計データの情報を損失関数に取り入れ、ミニバッチ学習をすることにある。
それによって、２つの世界を明示的に融合させることに成功した。